# Grammar and Spelling

- count the number of corrected words per user
- normalized by the total count of words by the user


In [1]:
import pandas as pd
import sqlite3

conn = sqlite3.connect('../../giicg.db')

prompts = pd.read_sql("SELECT ep.*, u.age, u.work_exp_years FROM translated_prompts ep JOIN users u ON ep.user_id = u.user_id", conn)
prompts

,message_id,conversation_id,role,message_text,conversational,code,other,gender,user_id,language,age,work_exp_years
0,1,1,user,"parsing data from python iterator, how it coul...","parsing data from python iterator, how it coul...",,,Man (cisgender),6,en,19-25,3.0
1,730,32,user,Write python function to do operations with in...,Write python function to do operations with in...,,report_dt\tsource\tmetric_name\tmetric_num\tme...,Man (cisgender),6,en,19-25,3.0
2,1133,55,user,Write shortest tutorial on creating RAG on ema...,Write shortest tutorial on creating RAG on ema...,,,Man (cisgender),6,en,19-25,3.0
3,1135,55,user,what is FAISS,what is FAISS,,,Man (cisgender),6,en,19-25,3.0
4,1137,55,user,Transform given code to process large .mbox file,Transform given code to process large .mbox file,,Transform given code to process large .mbox file,Man (cisgender),6,en,19-25,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...
750,1646,82,user,"def run_query(query, n_results):\n query_em...",this is my code. I want to: Get nodes and edge...,"def run_query(query, n_results):\n query_em...",,Man (cisgender),92,en,36-40,3.0
751,1845,37,user,\n nun möchte ich judgement balancing m...,Now I want to bring judgement balancing into t...,,,Woman (cisgender),29,de,26-30,None
752,1847,37,user,\n ich sehe keine veränderung im Plot. Was ...,I don't see any change in the plot.,,,Woman (cisgender),29,de,26-30,None
753,1849,2,user,\n I am working on the problem of reconstru...,\n I am working on the problem of reconstru...,,Classic CV - Drone navigation\nIf you ever tho...,Man (cisgender),8,en,19-25,1.0


In [2]:
user_prompts = (
    prompts
    .fillna({'work_exp_years': 'None'})
    .groupby(['user_id', 'gender', 'age', 'work_exp_years', 'language'])['message_text']
    .apply(' '.join)
    .reset_index()    # Reset index to create a DataFrame
)
user_prompts['n_words'] = user_prompts['message_text'].apply(lambda x: len(x.split()))

user_prompts

,user_id,gender,age,work_exp_years,language,message_text,n_words
0,6,Man (cisgender),19-25,3.0,en,"parsing data from python iterator, how it coul...",209
1,8,Man (cisgender),19-25,1.0,en,\n I am working on the problem of reconstru...,267
2,11,Woman (cisgender),26-30,1.0,en,Can you adapt the following code so that inste...,1406
3,15,Man (cisgender),26-30,6.0,en,\n SET_ALL_TABLES action is currently not f...,358
4,16,Woman (cisgender),19-25,None,en,I want to use Dummy Hot encoding to replace th...,3293
5,25,Man (cisgender),26-30,6.0,en,whats the best way to encode and compress a ja...,107
6,28,Woman (cisgender),31-35,5.0,en,I have a pandas dataframe like this:\ndata\tpe...,1826
7,29,Woman (cisgender),26-30,None,de,\n nun möchte ich judgement balancing m...,64
8,30,Non-binary,26-30,None,en,"as a NLP and LLM researcher, I am recently dow...",92
9,31,Man (cisgender),36-40,22.0,en,How can I make use of an ObservableHQDatabaseC...,430


In [36]:
corrections = {
    'orthogogal': 'orthogonal',
    'Anothers' : 'Another',
    'follwoing' : 'following',
    'insted' : 'instead',
    'addtion' : 'addition',
    'develope' : 'develop',
    'familys' : 'families',
    'responsable' : 'responsible',
    'optimate' : 'optimize',
    'esier' : 'easier',
    'sentencs': 'sentences',
    'preveent' : 'prevent',
    'indexs': 'indices',
    'hundrets': 'hundreds',
    'orthotogonal' : 'orthogonal',
    'palatte' : 'plaette',
    'favoriting' : 'favoring',
    'orignal' : 'original',
    'resemblens' : 'resemblance',
    'appoinment' : 'appointment',
    'ealier' : 'earlier',
    'Rectlange' : 'Rectangle',
    'gesorächslauf' : 'chat history',
    'evry': 'every',
    'inlcude': 'include',
    'wrtie' : 'write',
    'tring' : 'trying',
    'exaple' : 'example',
    'modalitieees': 'modalities',
    'doenst' : 'doesn\'t',
    'actualy': 'actually',
    'paremeter': 'parameter',
    'experiement': 'experiment',
    'concatination' : 'concatenation',
    'impliment' : 'implement',
    'impilmentation' : 'implementation',
    'nessacary' : 'necessary',
    'caclulated' : 'calculated',
    'matrixes' : 'matrices',
    'writie' : 'write',
    'follws' : 'follows',
    'inlcuded' : 'included',
    'origional' : 'original',
    'thta' : 'that',
    'laike' : 'like',
    'wehre' : 'where',
    'reuslts':'results',
    'intersetd' : 'interested',
    'concatition' : 'concatenation',
    'paramters' : 'parameters',
    'implyin' : 'implying',
    'guassion' : 'gaussian',
    'thigns' : 'things',
    'dockerfule' : 'dockerfile',
    'snipped'  : 'snippet',
    'viseos' : 'videos',
    'phne' : 'without',
    'deday' : 'decay',
    'seperate' : 'separate',
    'apporach' : 'approach',
    'indiviually' : 'individually',
    'allegeded' : 'alleged',
    'testset' : 'test set',
    'classificaiton' : 'classification',
    'optimice' : 'optimize',
    'adaptbefore' : 'adapt before',
    'onlz' : 'only',
    'sepearte' : 'separate',
    'informaiton' : 'information',
    'validaiton' : 'validation',
    'overfitiing' : 'overfitting',
    'eahc' : 'each',
    'directorys' : 'directories',
    'direcrtorys' : 'directories',
    'manipultaet' : 'manipulated',
    'aere' : 'are',
    'withoud' : 'without',
    'thorugh' : 'thorugh',
    'please provide example hot to use' : 'please provide example how to use',
    'rewertie' : 'rewrite',
    'inforamtion' : 'information',
    'visualiysing' : 'visualizing',
    'achive' : 'achieve',
    'sohuellete' : 'silhouette',
    'autimatically' : 'automatically',
    'entrys' : 'entries',
    'visualisaitons' : 'visualizations',
    'seachr' : 'search',
    'searhc' : 'search',
    'shoulndt' : 'shouldn\'t',
    'llookng' : 'looking',
    'clutering' : 'clustering',
    'restuks' : 'results',
    'mehtod' : 'method',
    'delimimetr' : 'delimiter',
    'strucuture' : 'structure',
    'neccessary' : 'necessary',
    'extractzion' : 'extraction',
    'suddently' : 'suddenly',
    'miminal' : 'minimal',
    'anomalious' : 'anomalous',
    'staistical' : 'statistical',
    'vlaues' : 'values',
    'calsses' : 'classes',
    'ommitted' : 'omitted',
    'approcheas' : 'approaches',
    'inthe' : 'in the',
    'sceintificly' : 'scientifically',
    'soonn' : 'soon',
    'dissappear' : 'disappear',
    'tryping' : 'trying',
    'undefinded' : 'undefined',
    'lenght' : 'length',
    'overritten' : 'overwritten',
    'exisiting' : 'existing',
    'caluclate' : 'calculate',
    'optimise' : 'optimize',
    'HEader' : 'Header',
    'nich tunterstrichen' : 'no underscore',
    'carussel': 'carousel',
    'adress' : 'address',
    'blancs' : 'blanks',
    'formated' : 'formatted',
    'unmanipluated': 'unmanipulated',
    'additonal': 'additional',

}



In [37]:
import re

# Pre-compile regex for each misspelling for word boundary matching
misspell_regex = {mis: re.compile(r'\b{}\b'.format(re.escape(mis)), flags=re.IGNORECASE)
                  for mis in corrections.keys()}

print(misspell_regex)

def count_misspells(text):
    """Count the total number of misspelled words in a text based on corrections dict."""
    return sum(len(regex.findall(text)) for regex in misspell_regex.values())

user_prompts['n_misspells'] = user_prompts['conversational'].apply(count_misspells)
user_prompts['misspell_rate'] = user_prompts['n_misspells'] / user_prompts['n_words']

# Display results
user_prompts[['user_id', 'gender', 'n_words', 'n_misspells', 'misspell_rate', 'conversational']]

{'orthogogal': re.compile('\\borthogogal\\b', re.IGNORECASE), 'Anothers': re.compile('\\bAnothers\\b', re.IGNORECASE), 'follwoing': re.compile('\\bfollwoing\\b', re.IGNORECASE), 'insted': re.compile('\\binsted\\b', re.IGNORECASE), 'addtion': re.compile('\\baddtion\\b', re.IGNORECASE), 'develope': re.compile('\\bdevelope\\b', re.IGNORECASE), 'familys': re.compile('\\bfamilys\\b', re.IGNORECASE), 'responsable': re.compile('\\bresponsable\\b', re.IGNORECASE), 'optimate': re.compile('\\boptimate\\b', re.IGNORECASE), 'esier': re.compile('\\besier\\b', re.IGNORECASE), 'sentencs': re.compile('\\bsentencs\\b', re.IGNORECASE), 'preveent': re.compile('\\bpreveent\\b', re.IGNORECASE), 'indexs': re.compile('\\bindexs\\b', re.IGNORECASE), 'hundrets': re.compile('\\bhundrets\\b', re.IGNORECASE), 'orthotogonal': re.compile('\\borthotogonal\\b', re.IGNORECASE), 'palatte': re.compile('\\bpalatte\\b', re.IGNORECASE), 'favoriting': re.compile('\\bfavoriting\\b', re.IGNORECASE), 'orignal': re.compile('\\b

,user_id,gender,n_words,n_misspells,misspell_rate,conversational
0,6,Man (cisgender),196,1,0.005102,"parsing data from python iterator, how it coul..."
1,8,Man (cisgender),36,0,0.000000,\n I am working on the problem of reconstru...
2,11,Woman (cisgender),193,0,0.000000,Can you adapt the following code so that inste...
3,15,Man (cisgender),36,0,0.000000,SET_ALL_TABLES action is currently not fetchin...
4,16,Woman (cisgender),542,7,0.012915,I want to use Dummy Hot encoding to replace th...
5,25,Man (cisgender),107,1,0.009346,whats the best way to encode and compress a ja...
6,28,Woman (cisgender),561,5,0.008913,I have a pandas dataframe like this:\n\nI want...
7,29,Woman (cisgender),64,0,0.000000,\n nun möchte ich judgement balancing m...
8,30,Non-binary,92,0,0.000000,"as a NLP and LLM researcher, I am recently dow..."
9,31,Man (cisgender),92,0,0.000000,How can I make use of an ObservableHQDatabaseC...


In [38]:
gender_means = user_prompts.groupby('gender')['misspell_rate'].mean()
gender_means

gender
Another gender       0.007937
Man (cisgender)      0.005004
Non-binary           0.000000
Woman (cisgender)    0.014126
Name: misspell_rate, dtype: float64

In [39]:
from statsmodels.stats.multitest import multipletests
from helpers.statistical_tests import compare_genders


def groupwise_comparison_with_correction(df, columns):
    stat_vals = []
    p_vals = []
    test_types = []
    female_vars = []
    male_vars = []
    nonzero_females = []
    nonzero_males = []
    for col in columns:
        stat_value, p_value, test_type, female_var, male_var, nonzero_female, nonzero_male = compare_genders(df, col)
        stat_vals.append(stat_value)
        p_vals.append(p_value)
        test_types.append(test_type)
        female_vars.append(female_var)
        male_vars.append(male_var)
        nonzero_females.append(nonzero_female)
        nonzero_males.append(nonzero_male)
    reject, pvals_corrected, _, _ = multipletests(p_vals, alpha=0.05, method='fdr_bh')

    results = pd.DataFrame({
        'word': columns,
        'test type': test_types,
        'stat_value': stat_vals,
        'p_value': p_vals,
        'corrected p_value': pvals_corrected,
        'f_var': female_vars,
        'm_var': male_vars,
        'n_hits': [{'female': f, 'male': m} for f, m in zip(nonzero_females, nonzero_males)]
    })
    return results

In [40]:
results = groupwise_comparison_with_correction(user_prompts, ['misspell_rate'])
results

,word,test type,stat_value,p_value,corrected p_value,f_var,m_var,n_hits
0,misspell_rate,U-test,74.0,0.274431,0.274431,0.000424,0.000035,"{'female': 9, 'male': 8}"


In [41]:
def print_misspelled_matches(text):
    """Print all substrings in the text that match any misspelling pattern."""
    matches = []
    for mis, regex in misspell_regex.items():
        found = regex.findall(text)
        if found:
            matches.extend(found)
    print(matches)


user_65 = user_prompts[user_prompts['user_id'] == 65]
user_65['conversational'].apply(print_misspelled_matches)


['phne']


18    None
Name: conversational, dtype: object

In [3]:
user_prompts_en = user_prompts[user_prompts['language'] == 'en']
user_prompts_de = user_prompts[user_prompts['language'] == 'de']
user_prompts_it = user_prompts[user_prompts['language'] == 'it']

In [5]:
import language_tool_python

#tool_de = language_tool_python.LanguageTool('de')
#tool_it = language_tool_python.LanguageTool('it')

def count_grammar_mistakes(text, tool):
    spelling_mistakes = 0
    grammar_mistakes = 0
    matches = tool.check(text)
    for match in matches:
        if getattr(match, "ruleIssueType", None) == "misspelling":
            spelling_mistakes += 1
        else:
            grammar_mistakes += 1
    return spelling_mistakes, grammar_mistakes

with language_tool_python.LanguageToolPublicAPI('en-US') as tool:
    user_prompts_en[["n_typos", "n_grammar_errors"]] = user_prompts_en["message_text"].apply(lambda x: pd.Series(count_grammar_mistakes(x, tool)))



LanguageToolError: <!DOCTYPE html>
<!--[if lt IE 7]> <html class="no-js ie6 oldie" lang="en-US"> <![endif]-->
<!--[if IE 7]>    <html class="no-js ie7 oldie" lang="en-US"> <![endif]-->
<!--[if IE 8]>    <html class="no-js ie8 oldie" lang="en-US"> <![endif]-->
<!--[if gt IE 8]><!--> <html class="no-js" lang="en-US"> <!--<![endif]-->
<head>


<title>languagetool.org | 520: Web server is returning an unknown error</title>
<meta charset="UTF-8" />
<meta http-equiv="Content-Type" content="text/html; charset=UTF-8" />
<meta http-equiv="X-UA-Compatible" content="IE=Edge" />
<meta name="robots" content="noindex, nofollow" />
<meta name="viewport" content="width=device-width,initial-scale=1" />
<link rel="stylesheet" id="cf_styles-css" href="/cdn-cgi/styles/main.css" />


</head>
<body>
<div id="cf-wrapper">
    <div id="cf-error-details" class="p-0">
        <header class="mx-auto pt-10 lg:pt-6 lg:px-8 w-240 lg:w-full mb-8">
            <h1 class="inline-block sm:block sm:mb-2 font-light text-

In [ ]:
user_prompts_en